# Register

## Networks

### Display the registered networks

To know which networks parameter are available, use the function
`available_networks`.

In [1]:
from harissa.benchmark import available_networks

print(sorted(available_networks()))

['BN8', 'CN5', 'FN4', 'FN8', 'Trees10', 'Trees100', 'Trees20', 'Trees5', 'Trees50']


## Register networks parameter

To register your custom Inference method:

1.  Implement a function that creates a `harissa.core.NetworkParameter`. 
For more details see the [harissa's networks](https://github.com/harissa-framework/harissa/tree/main/src/harissa/networks).

    ```python
    def repressilator():
        n = NetworkParameter(3)
        
        # Basal gene activities
        n.basal[1] = 5
        n.basal[2] = 5
        n.basal[3] = 5
        
        # Inhibitions in cycle
        n.interaction[1,2] = -10
        n.interaction[2,3] = -10
        n.interaction[3,1] = -10

        # Degradation rates (per unit of time)
        n.degradation_rna[:] = 1.0
        n.degradation_protein[:] = 0.2

        n.burst_frequency_min[:] = 0.0 * n.degradation_rna
        n.burst_frequency_max[:] = 2.0 * n.degradation_rna

        # Creation rates
        n.creation_rna[:] = n.degradation_rna * n.rna_scale() 
        n.creation_protein[:] = n.degradation_protein * n.protein_scale()

        return n
    ```

2.  Register it with the `NetworksGenerator.register()` method.

In [2]:
from harissa import NetworkParameter
from harissa.benchmark.generators import NetworksGenerator

def repressilator():
    n = NetworkParameter(3)
    
    # Basal gene activities
    n.basal[1] = 5
    n.basal[2] = 5
    n.basal[3] = 5
    
    # Inhibitions in cycle
    n.interaction[1,2] = -10
    n.interaction[2,3] = -10
    n.interaction[3,1] = -10

    # Degradation rates (per unit of time)
    n.degradation_rna[:] = 1.0
    n.degradation_protein[:] = 0.2

    n.burst_frequency_min[:] = 0.0 * n.degradation_rna
    n.burst_frequency_max[:] = 2.0 * n.degradation_rna

    # Creation rates
    n.creation_rna[:] = n.degradation_rna * n.rna_scale() 
    n.creation_protein[:] = n.degradation_protein * n.protein_scale()

    return n

NetworksGenerator.register(
    name='RN3',
    network=repressilator
)

print(sorted(available_networks()))

['BN8', 'CN5', 'FN4', 'FN8', 'RN3', 'Trees10', 'Trees100', 'Trees20', 'Trees5', 'Trees50']


## Inferences

### Display the registered inference methods

To know which inference methods are available, use the function `available_inferences`.

In [3]:
from harissa.benchmark import available_inferences

print(sorted(available_inferences()))

['Cardamom', 'Hartree', 'Pearson']


### Register inference methods

To register your custom Inference method:

1.  Implement a subclass of `harissa.core.Inference`. 
For more details see the [harissa's inferences](https://github.com/harissa-framework/harissa/tree/main/src/harissa/inference) or the implementations examples inside the [examples folder](https://github.com/harissa-framework/benchmark/tree/main/examples).

    ```python
    class Genie3(Inference):
        @property
        def directed(self):
            return True
        
        def run(self,
            data: Dataset,
            param: NetworkParameter
        ) -> Inference.Result:
            param.interaction[:] = genie3(data.count_matrix)
            return Inference.Result(param)
    ```

2.  Register it with the `InferencesGenerator.register()` 
    method along with its name and its colors 
    (used for the plots inside the reports). 
    `InferencesGenerator` provides a color map but your custom colors by passing a tuple of RGBA values.

In [4]:
import sys
sys.path.append('../examples')
import numpy as np

from harissa.benchmark.generators import InferencesGenerator


from genie3 import Genie3
from sincerities import Sincerities

InferencesGenerator.register(
    name='Genie3', 
    inference=Genie3,
    colors=np.array([
        InferencesGenerator.color_map(0), 
        InferencesGenerator.color_map(1)
    ])
    
)

InferencesGenerator.register(
    name='Sincerities', 
    inference=Sincerities,
    colors=np.array([
        InferencesGenerator.color_map(2),
        InferencesGenerator.color_map(3)
    ])
)

print(sorted(available_inferences()))

['Cardamom', 'Genie3', 'Hartree', 'Pearson', 'Sincerities']
